In [1]:
from packages import Exam
from packages import Unit
from packages.Unit import N, m, m2, m3, m4, pers, Pa,  K
import numpy as np

path = ["./files/result/csv/top_sample.csv",
        "./files/result/csv/back_sample.csv"]
unit_instances = [N(), m(), m2(), m3(), m4(), pers(), Pa(), K()]

exam = Exam.Exam(path)
exam.get_marks()
exam.get_student()
#q1
exam.parts.append(Exam.SingleAlphabetPart(marks=exam.marks[0][4:9, 1:27], corrects=["A","Q","B","H","P"], allocations=[2] * 5))
#q2
exam.parts.append(Exam.SingleAlphabetPart(marks=exam.marks[0][10:20, 1:27], corrects=["A","E","B","C","H","I","J","M","N","O"], allocations=[2] * 10))
#q3
exam.parts.append(Exam.DualNumberPart(
    marks = np.vstack([exam.marks[0][21:37, 1:27], exam.marks[0][2:6, 29:55]]),
    unit_instances=unit_instances,
    corrects = [
        [{"value":1e3, "unit":Unit.N()}, {"value":-3e3, "unit":Unit.Nm()}],
        [{"value":3e2, "unit":Unit.N()}, {"value":-5e2, "unit":Unit.Nm()}],
        [{"value":4e1, "unit":Unit.N()}, {"value":-4e1, "unit":Unit.N()}],
        [{"value":1e3, "unit":Unit.N()}, {"value":1e3, "unit":Unit.N()}],
        [{"value":-6e1, "unit":Unit.N()}, {"value":5e2, "unit":Unit.N()}],
    ],
    allocations = [6] * 5
    ))
#q4
variables = [
    {"name":"i", "equation":"np.linspace(0,1,100)", "unit":Unit.non(), "first difine": 0},
    {"name":"y", "equation":"np.linspace(1,2,100)", "unit":Unit.m(), "first difine": 0},
    {"name":"x", "equation":"np.linspace(0, 2 / L , 10)", "unit" : Unit.m(), "first difine": 0},
    {"name":"EI", "value":2.5e5, "unit": Unit.Nm2(), "first difine": 0},
    {"name":"h", "value":20e-3, "unit": Unit.m(), "first difine": 0},
    {"name":"b", "value":30e-3, "unit": Unit.m(), "first difine": 0},
    {"name":"w0", "value":1e2, "unit": Unit.N_per_m(), "first difine": 0},
    {"name":"L", "value":1e1, "unit": Unit.m(), "first difine": 0},
    {"name":"L^2", "equation":" L ** 2", "unit": Unit.m2(), "first difine": 0},
    {"name":"L^3", "equation":" L ** 3", "unit": Unit.m3(), "first difine": 0},
    {"name":"L^4", "equation":" L ** 4", "unit": Unit.m4(), "first difine": 0}
]

corrects = [
    {"name":"RA", "equation":" w0 * L "},
    {"name":"RB", "equation":" w0 * L "},
    {"name":"a", "equation":" w0 * x / L "},
    {"name":"x1", "equation":" y "},
    {"name":"x2", "equation":" y / L "},
    {"name":"theta", "equation":" 5 * w0 * L ** 3 / 48 / EI "},
    {"name":"delta", "equation":" 13 * w0 * L ** 4 / 384 / EI "},    
    ]

question_classes = [
    Exam.EquationAndNumberQuestion,
    Exam.EquationAndNumberQuestion,
    Exam.EquationQuestion,
    Exam.EquationQuestion,
    Exam.EquationQuestion,
    Exam.EquationAndNumberQuestion,
    Exam.EquationAndNumberQuestion
    ]

exam.parts.append(
    Exam.Part(
        marks = np.vstack([exam.marks[0][7:37, 29:55], exam.marks[1][2:7, 1:27]]),
        variables = variables,
        corrects = corrects,
        allocations = [2] * 5 + [5] * 2,
        question_classes=question_classes,
        unit_instances=unit_instances
        )
    )


exam.scoring()

print(exam.score)
 

scoring start of Q 1
scoring start of Q 2
scoring start of Q 3
correct: 1000.0 N  ===== answer: 1000.0 N
correct: -3000.0 Nm  ===== answer: -3000.0 Nm
correct: -3000.0 Nm  ===== answer: 1000.0 N
correct: 1000.0 N  ===== answer: -3000.0 Nm
correct: 1000.0 N  ===== answer: 1000.0 N
correct: -3000.0 Nm  ===== answer: -3000.0 Nm
correct: 300.0 N  ===== answer: 300.0 N
correct: -500.0 Nm  ===== answer: -500.0 Nm
correct: -500.0 Nm  ===== answer: 300.0 N
correct: 300.0 N  ===== answer: -500.0 Nm
correct: 300.0 N  ===== answer: 300.0 N
correct: -500.0 Nm  ===== answer: -500.0 Nm
correct: 40.0 N  ===== answer: 40.0 N
correct: -40.0 N  ===== answer: -40.0 N
correct: -40.0 N  ===== answer: 40.0 N
correct: 40.0 N  ===== answer: -40.0 N
correct: 40.0 N  ===== answer: 40.0 N
correct: -40.0 N  ===== answer: -40.0 N
correct: 1000.0 N  ===== answer: 1000.0 N
correct: 1000.0 N  ===== answer: 1000.0 N
correct: 1000.0 N  ===== answer: 1000.0 N
correct: 1000.0 N  ===== answer: 1000.0 N
correct: 1000.0 N  

In [2]:
for p in exam.parts:
    print(p.unit_instances)
    print(p.score)

[]
10
[]
20
[N, m, m2, m3, m4, /s, Pa, K]
30.0
[N, m, m2, m3, m4, /s, Pa, K]
15.5


In [3]:
for p in exam.parts:
    print(p.score)

10
20
30.0
15.5


q4

In [4]:
for p in exam.parts[3].questions:    
    print(p.score)

2.0
2.0
0
2
2
5.0
2.5


In [5]:
exam.parts[3].questions[0].answers[0].score

1.0

In [6]:
from packages import Marksheet

marker_positions = [
    ( 2, 26),
    ( 2, 28),
    ( 2, 56),
    ( 3, 24),
    (40,  2),
    (40, 56),
]

sheet = Marksheet(path = "./files/scan/back_sample.jpg", 
                  row = 41, column = 57, 
                  marker_positions = marker_positions,
                  binary_threshold=240,
                  monitor= True)
sheet.read()
sheet.rotation()
sheet.aliment()
sheet.threshold_check()
sheet.make_grids()
sheet.mark_check()
sheet.draw_grid()
sheet.save()

init
open ./files/scan/back_sample.jpg
read image shape: (2341, 1643, 3)
rotated to horizontal
make reference image shape: (1562, 2258) reqtangle h,w: 40.073170731707314 41.07017543859649
rotation: 0, max_val: 0.17581678926944733
rotation: 1, max_val: 0.5526888370513916
best_rotation: 1
rotation complete
aliment start
make reference image shape: (1562, 2258) reqtangle h,w: 40.073170731707314 41.07017543859649


In [ ]:
from packages import Marksheet

marker_positions = [
    ( 2, 28),
    ( 2, 56),
    (40,  2),
    (40,  4),
    (40, 56),
]

sheet = Marksheet(path = "./files/scan/top_sample.jpg", 
                  row = 41, column = 57, 
                  marker_positions=marker_positions,
                  binary_threshold=240,
                  monitor= True)

sheet.read()
sheet.rotation()
sheet.aliment()
sheet.draw_grid()
sheet.threshold_check()
sheet.make_grids()
sheet.mark_check()
sheet.save()
